<a href="https://colab.research.google.com/github/Inostroza7/OpenAI-LangChain-Basics/blob/main/OpenAI_LangChain_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤖🌟 OpenAI & 🌟🦜️🔗 LangChain 📚 Basics

## 🌈 Descripción 📚

Este curso te proporciona las habilidades esenciales para dominar tanto la API de OpenAI como el ecosistema de LangChain.
A través de una combinación de teoría y práctica, te prepararemos para implementar soluciones de vanguardia en inteligencia artificial. 🚀

## 🛠 Requisitos ⚙️

- Python 3.x 🐍
- Acceso a la API de OpenAI 🤖
- Conocimientos básicos de programación 💻

## 🚀 ¿Listo para empezar? 🌟

¡Vamos allá! 💥

### 🤖 API de OpenAI

#### Instalar OpenAI 🌐

In [ ]:
pip install openai

#### Importa la libreria 📚

In [ ]:
import openai

#### Configuración del entorno 🛠️

In [ ]:
openai.api_key = 'Ingresa aquí tu API Key de OpenAI'

#### Tu primer Chatbot 👋

Ingresa tu prompt

In [ ]:
prompt = 'Hola Chatbot'

- System: contexto de fondo útil que le indica a la IA qué hacer
- Human: mensajes que pretenden representar al usuario.
- AI: mensajes que muestran con qué respondió la IA.

In [ ]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt},],
    temperature=1
)

print(response)

In [ ]:
response.choices[0].message['content']

### 🦜️🔗 LangChain

#### Instalar Langchain 🌐

In [ ]:
pip install langchain

#### LLM (LangChain Language Model) 🗨️

Define tu LLM

In [ ]:
from langchain.llms import OpenAI
llm = OpenAI(temperature=0.3)

Prueba tu LLM

In [ ]:
prompt = 'Hola, cómo estás??'
print(llm(prompt))

Importa las librerias del chat

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage

In [ ]:
prompt = 'Hola cómo estás?'
chat = ChatOpenAI(model='gpt-3.5-turbo',temperature=0.3)
messages = [
    SystemMessage(content='You are a helpful assistant'),
    HumanMessage(content=prompt)
]
response = chat(messages)
print(response)

Obtén sólo el texto

In [ ]:
response.content

#### Embeddings 📖

Definimos la función para cargar el documento

In [ ]:
def load_document(file):
    import os
    name, extension = os.path.splitext(file)

    if extension == '.pdf':
        from langchain.document_loaders import PyPDFLoader
        print(f'Loading {file}')
        loader = PyPDFLoader(file)
    elif extension == '.docx':
        from langchain.document_loaders import Docx2txtLoader
        print(f'Loading {file}')
        loader = Docx2txtLoader(file)
    elif extension == '.txt':
        from langchain.document_loaders import TextLoader
        loader = TextLoader(file)
    else:
        print('El formato del archivo no es válido')
        return None

    data = loader.load()
    return data

Definimos la función para hacer los Chunks

In [ ]:
def chunk_data(data, chunk_size=500, chunk_overlap=0):
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = text_splitter.split_documents(data)
    return chunks

Definimos la función para crear los embeddings

In [ ]:
def create_embeddings(chunks):
    from langchain.embeddings.openai import OpenAIEmbeddings
    embeddings = OpenAIEmbeddings()

    from langchain.vectorstores import Chroma
    vector_store = Chroma.from_documents(chunks, embeddings)
    return vector_store

Definimos nuestro Chat

In [ ]:
def ask_and_get_answer(vector_store, q, k=3):
    from langchain.chains import RetrievalQA
    from langchain.chat_models import ChatOpenAI

    llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.2)
    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': k})
    chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

    answer = chain.run(q)
    return answer

Carga tu PDF

In [ ]:
data = load_document('Oppenheimer.pdf')
print(f'El documento tiene {len(data)} paginas')

In [ ]:
chunks = chunk_data(data)
print(len(chunks))

Crea los embeddings

In [ ]:
vector_store = create_embeddings(chunks)

Prueba el Chat

In [ ]:

q = 'Quien protagonizó la película Oppenhemier'
k = 3
answer = ask_and_get_answer(vector_store, q, k)
print(answer)

In [ ]:
import time

i = 1
print('Escribe Chao, Salir o Adios para salir.')
while True:
    q = input(f'Pregunta #{i}: ')
    i = i+1
    if q.lower() in  ['salir', 'chao', 'adios']:
        print('Hasta luego!')
        time.sleep(2)
        break

    answer = ask_and_get_answer(vector_store, q, 5)
    print(answer)
    print(f'\n {"-"*50} \n')

#### Memoria 🧠

Definimos nuestro contador de Tokens

In [ ]:
from langchain.callbacks import get_openai_callback

def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Has utilizado {cb.total_tokens} tokens')

    return result

Definimos nuestro modelo y cadena

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain

llm = ChatOpenAI(
	temperature=0,
	model_name="gpt-3.5-turbo"
)

conversation = ConversationChain(llm=llm)

ConversationBufferMemory

In [ ]:
from langchain.chains.conversation.memory import ConversationBufferMemory

conversation_buf = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

In [ ]:
conv_buf = conversation_buf('Hola, cómo estás?')
print(conv_buf)

In [ ]:
count_tokens(conversation_buf, 'Que artistas tienen mi nombre?')

ConversationBufferWindowMemory

In [ ]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

conversation_bufw = ConversationChain(
	llm=llm,
	memory=ConversationBufferWindowMemory(k=2)
)

In [ ]:
count_tokens(conversation_bufw,"Cuál es mi nombre!")

In [ ]:
bufw_history = conversation_bufw.memory.load_memory_variables(
    inputs=[]
)['history']
print(bufw_history)

#### Agentes 🕵️

Definimos el LLM

In [ ]:
from langchain import OpenAI

llm = OpenAI(
    temperature=0,
    model_name="gpt-3.5-turbo"
)

Definimos nuestra Tool de Calculadora

In [ ]:
from langchain.chains import LLMMathChain
from langchain.agents import Tool

llm_math = LLMMathChain(llm=llm)

math_tool = Tool(
    name='Calculator',
    func=llm_math.run,
    description='Useful for when you need to answer questions about math.')

tools = [math_tool]

Iniciamos nuestro agente

In [ ]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3
)

Le preguntamos nuestro cálculo

In [ ]:
zero_shot_agent("Cuanto es(4.5*2.1)^2.2?")

Prueba el agente con un problema matemático

In [ ]:
zero_shot_agent("Si María tiene cuatro manzanas y Jorge trae dos cajas y media de manzanas"
                "(cada caja tiene 8 manzanas) cuantas manzanas tendrá María en total?")

#### Tools 🛠️

Instalamos el módulo

In [ ]:
pip install duckduckgo-search

Definimos nuestra herramienta

In [ ]:
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

searchTool = Tool(
                name="Intermediate Answer",
                func=search.run,
                description="useful for when you need to ask with search",
            )

Creamos nuestro agente

In [ ]:
from langchain.agents import load_tools
from langchain.utilities import SerpAPIWrapper

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=[searchTool,math_tool],
    llm=llm,
    verbose=True,
    max_iterations=5
)

Probamos nuestro agente

In [ ]:
zero_shot_agent("Cual es el sueldo actual de Cristiano Ronaldo en pesos chilenos?")